#第一次分组作业
*8组小组成员：邱子源、张翥、夏英凡、李岱遥

*行业：核电

*分工：统一

In [14]:
import sqlite3
con= sqlite3.connect('G:\Anaconda/teamwork5.db')  

*建立数据库

In [2]:
import pandas.io.sql as sql
from lxml.html import parse
from urllib2 import urlopen
from os.path import getsize
import pandas as pd
import urllib 
import numpy as np
import os
import sys
def get_sina_stock(stockid,start="1990-12-19",end=pd.datetime.now(),isstock=True,fuquan=True):    
    def _unpack(row, kind='td'):
        elts = row.findall('.//%s' % kind)
        return [val.text_content().strip() for val in elts] # .strip()鍘绘帀\r\t\n涔嬬被鐨勫瓧绗�    
    def parse_options_data(table,isstock,fuquan):
        rows = table.findall('.//tr')
        data = [_unpack(r) for r in rows[2:]]
        if isstock & fuquan:
            colnames = ['date','open','high','close','low','vol','amount','ratio'] # 鍥為伩涓枃澶勭悊
        else:
            colnames = ['date','open','high','close','low','vol','amount']
        data= pd.DataFrame(data,columns=colnames)
        data.index=pd.to_datetime(data["date"])
        data=data.drop("date",axis=1)
        data=data.ix[range(len(data)-1,-1,-1)]
        return  data
    data=pd.DataFrame()
    daterange=pd.date_range(start,end,freq="Q")
    daterange=daterange.insert(len(daterange),daterange[-1]+1)
    for cq in daterange: 
        if isstock:
            if fuquan:
                url='http://vip.stock.finance.sina.com.cn/corp/go.php/vMS_FuQuanMarketHistory/stockid/'+str(stockid)+'.phtml?year='+str(cq.year)+'&jidu='+str(cq.month/3)
            else:
                url='http://vip.stock.finance.sina.com.cn/corp/go.php/vMS_MarketHistory/stockid/'+str(stockid)+'.phtml?year='+str(cq.year)+'&jidu='+str(cq.month/3)            
        else:            
            url='http://vip.stock.finance.sina.com.cn/corp/go.php/vMS_MarketHistory/stockid/'+str(stockid)+'/type/S.phtml?year='+str(cq.year)+'&jidu='+str(cq.month/3)
        try:
            parsed = parse(urlopen(url))
        except:
            print "download failed for year=" +str(cq.year)+',  jidu='+str(cq.month/3)
            continue
        doc = parsed.getroot()
        tables=doc.findall('.//table')     
        da= tables[-1] # last table 19 for fuquan data and 4 for index
        datatem = parse_options_data(da,isstock,fuquan)
        data=pd.concat([data,datatem])
    return pd.DataFrame(data[start:end],dtype=float)    

*下载日线数据

In [3]:
def get_sina_high(stockid,start,end):
    #stockid with pre sh or sz
    data=pd.DataFrame()
    daterange=pd.date_range(start,end,freq="B")
    for cq in daterange:
        url='http://market.finance.sina.com.cn/downxls.php?date='+str(cq.date())+'&symbol='+stockid
        urllib.urlretrieve(url, "temp.txt")    
        # if no data, temp.txt just contains few words        
        if getsize("temp.txt")<100:
            continue
        HighF=pd.read_table("temp.txt",na_values="--",encoding="gbk",index_col=0,parse_dates=True)
        HighF=HighF.ix[range(len(HighF)-1,-1,-1)]
        delta=cq.date()-HighF.index.date[0]
        HighF.index=HighF.index+delta
        data=pd.concat([data,HighF])
    return data

*下载高频数据
高频数据网站需要一个英文前缀

In [4]:
def get_sina_fin(stockid,year):
    def _unpack(row,kind='td'):
        elts = row.findall('.//%s' % kind)
        return [val.text_content().strip() for val in elts] # .strip()鍘绘帀\r\t\n涔嬬被鐨勫瓧绗�    
    def parse_options_data(table):
        rows = table.findall('.//tr')
        data = [_unpack(r) for r in rows[1:]]    
        data= pd.DataFrame(data)
        
        datacol=pd.to_datetime(data.ix[0][1:])
        dataindex=data[0][1:]        
        data=data.drop(0,axis=1)
        data=data.ix[1:]
        data.index=dataindex
        data.columns=datacol
        return  data

    url="http://money.finance.sina.com.cn/corp/go.php/vFD_ProfitStatement/stockid/"+str(stockid)+"/ctrl/"+str(year)+"/displaytype/4.phtml"
    parsed = parse(urlopen(url))
    doc = parsed.getroot()
    tables=doc.findall('.//table')
    da= tables[-2]      
    datatem = parse_options_data(da)
     
    return datatem

*下载财务数据

In [5]:
IDlist1=['600089',
'600202',
'600226',
'600269',
'600292',
'600328',
'600416',
'600516',
'600550',
'600558',
'600622',
'600642',
'600848',
'600875',
'600973',
'601106',
'601179',
'601369',
'601727',
'601991',
'603100',
'603169',
'603308',
'603699',
'000777',
'000825',
'000883',
'000899',
'000922',
'000967',
'000993',
'002011',
'002058',
'002130',
'002167',
'002212',
'002227',
'002248',
'002255',
'002266',
'002318',
'002364',
'002366',
'002438',
'002478',
'002534',
'300004',
'300092',
'300153',
'300411'
]
IDlist2=['sh600089',
'sh600202',
'sh600226',
'sh600269',
'sh600292',
'sh600328',
'sh600416',
'sh600516',
'sh600550',
'sh600558',
'sh600622',
'sh600642',
'sh600848',
'sh600875',
'sh600973',
'sh601106',
'sh601179',
'sh601369',
'sh601727',
'sh601991',
'sh603100',
'sh603169',
'sh603308',
'sh603699',
'sz000777',
'sz000825',
'sz000883',
'sz000899',
'sz000922',
'sz000967',
'sz000993',
'sz002011',
'sz002058',
'sz002130',
'sz002167',
'sz002212',
'sz002227',
'sz002248',
'sz002255',
'sz002266',
'sz002318',
'sz002364',
'sz002366',
'sz002438',
'sz002478',
'sz002534',
'sz300004',
'sz300092',
'sz300153',
'sz300411'
]

In [6]:
for i in range(len(IDlist1)):
    x=get_sina_stock(IDlist1[i],'2010-11-20','2015-11-20')
    reload(sys)
    sys.setdefaultencoding( "utf-8" )
    sql.to_sql(x,'daily'+IDlist1[i],con)

*将日线数据导入数据库

In [7]:
for i in range(len(IDlist2)):
    try:
        x=get_sina_high(IDlist2[i],'2015-10-11','2015-10-18')
        reload(sys)
        sys.setdefaultencoding( "utf-8" )
        sql.to_sql(x,'high'+IDlist2[i],con)
    except:
        print(IDlist2[i])
        continue

*将高频数据导入数据库

In [8]:
from __future__ import division
import os
import sys
from urllib2 import urlopen
import urllib 
import numpy as np
from numpy.random import randn
from pandas import Series, DataFrame
import pandas as pd
import matplotlib.pyplot as plt
#import urllib.request
from lxml.html import parse

#from urllib.request import urlopen
from pandas.io.parsers import TextParser
import sqlite3

In [9]:
def _unpack(row, kind='td'):
    elts = row.findall('.//%s' % kind)
    return [val.text_content().strip() for val in elts]

def breakdata(url,stockname,tableinfo=19,dataamounts = 12,timelenth=23):
    parsed = parse(urlopen(url))
    doc = parsed.getroot()
    tables=doc.findall('.//table')
    table = tables[tableinfo]
    row = table.findall('.//tr')
    data = [_unpack(r) for r in row[1:]]
    flag = 0
    tot = (dataamounts+1)*timelenth+1
    for i in range(1,min(tot,len(row)),dataamounts+1):
        header=_unpack(row[i],kind="td")
        header = [stockname,header[1]]
        x = TextParser(data[i:i+dataamounts-1], names=header).get_chunk()
        x = x.set_index([header[0]])
        if flag ==0:
            y = x
            flag = 1
        else:
            y = y.join(x,how='outer')
    y = y.T
    return y

In [17]:
# need lab data from the website; 

def str6(x):
    return (6-len(str(x)))*'0'+str(x)
for nm in IDlist1:
    pp = str6(nm)
    url = 'http://vip.stock.finance.sina.com.cn/corp/go.php/vFD_FinanceSummary/stockid/'+pp+'.phtml'
    stockname = pp + 'caiwuzhaiyao'
    y = breakdata(url,stockname)
    print(y)
    try:
        y.to_sql(stockname,con)
    except:
        print('Table Name Overlap')

*财务数据导入数据库

In [15]:
mysql="SELECT * from daily600089 limit 5"
sql.read_sql(mysql,con)

,date,open,high,close,low,vol,amount,ratio
0,2010-11-22 00:00:00,436.836,444.348,435.470,432.738,27959724,538940288,22.764
1,2010-11-23 00:00:00,435.470,435.470,417.487,407.471,46460964,852544896,22.764
2,2010-11-24 00:00:00,410.885,426.592,425.226,410.885,39142340,724119808,22.764
3,2010-11-25 00:00:00,428.641,440.478,437.974,428.641,38646552,736474048,22.764
4,2010-11-26 00:00:00,436.608,444.120,437.747,431.828,31161028,599661504,22.764


*日线数据展示

In [16]:
mysql="SELECT * from highsh600089 limit 5"
sql.read_sql(mysql,con)

,成交时间,成交价,价格变动,成交量(手),成交额(元),性质
0,2015-10-12 09:25:07,11.19,11.19,4055,4537545,买盘
1,2015-10-12 09:30:08,11.23,0.04,2073,2328057,买盘
2,2015-10-12 09:30:13,11.23,NaN,136,152728,买盘
3,2015-10-12 09:30:23,11.25,0.02,103,115875,买盘
4,2015-10-12 09:30:28,11.23,-0.02,2,2246,中性盘


*高频数据展示

In [19]:
mysql="SELECT * from '600089caiwuzhaiyao'"
form = sql.read_sql(mysql,con)
form

,index,每股净资产,每股收益,每股现金含量,每股资本公积金,固定资产合计,流动资产合计,资产总计,长期负债合计,主营业务收入,财务费用,净利润
0,2015-09-30,6.3002元,0.497元,-0.2636元,2.5085元,None,4204460万元,6597820万元,1035950万元,2742020万元,47001.2万元,161473万元
1,2015-06-30,6.1576元,0.3417元,0.2341元,2.508元,None,4226150万元,6607980万元,1041880万元,1859390万元,35878.2万元,111034万元
2,2015-03-31,6.1856元,0.1695元,-0.5729元,2.4888元,None,3705290万元,6098720万元,1136020万元,853485万元,22067.4万元,54922.4万元
3,2014-12-31,6.0329元,0.5088元,-0.4264元,2.4888元,None,3512710万元,5929170万元,972430万元,3607480万元,64209.9万元,164860万元
4,2014-09-30,6.0369元,0.4257元,-1.0555元,2.5109元,None,3296390万元,5623820万元,981866万元,2647270万元,44727.4万元,137933万元
5,2014-06-30,5.9176元,0.2898元,-0.5141元,2.4668元,None,3150930万元,5433830万元,977215万元,1707060万元,25432.8万元,91743.2万元
6,2014-03-31,5.8717元,0.1381元,-1.0746元,2.447元,None,3124020万元,5413620万元,1026530万元,780186万元,13823.2万元,43723.1万元
7,2013-12-31,5.5402元,0.504元,0.6753元,1.7515元,None,2767310万元,5066020万元,1034860万元,2917470万元,52713.3万元,132838万元
8,2013-09-30,5.4262元,0.3888元,-1.6306元,1.7799元,None,2487450万元,4627590万元,1070560万元,1918490万元,31380.5万元,102479万元
9,2013-06-30,5.2734元,0.2561元,-1.015元,1.7536元,None,2223580万元,4365950万元,1044720万元,1150540万元,20247.2万元,67485.3万元
